##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 自動微分と勾配テープ

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/customization/autodiff"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ja/tutorials/customization/autodiff.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ja/tutorials/customization/autodiff.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/ja/tutorials/customization/autodiff.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Note: これらのドキュメントは私たちTensorFlowコミュニティが翻訳したものです。コミュニティによる 翻訳は**ベストエフォート**であるため、この翻訳が正確であることや[英語の公式ドキュメント](https://www.tensorflow.org/?hl=en)の 最新の状態を反映したものであることを保証することはできません。 この翻訳の品質を向上させるためのご意見をお持ちの方は、GitHubリポジトリ[tensorflow/docs](https://github.com/tensorflow/docs)にプルリクエストをお送りください。 コミュニティによる翻訳やレビューに参加していただける方は、 [docs-ja@tensorflow.org メーリングリスト](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ja)にご連絡ください。

これまでのチュートリアルでは、テンソルとその演算について紹介しました。このチュートリアルでは、機械学習モデルの最適化の鍵となる手法である[自動微分](https://ja.wikipedia.org/wiki/自動微分)を扱います。

## 設定

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

## 勾配テープ

TensorFlow には、自動微分、すなわち、入力変数に対する計算結果の勾配を計算するための[tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) API があります。TensorFlow は、`tf.GradientTape` のコンテキスト内で行われる演算すべてを「テープ」に「記録」します。その後 TensorFlow は、そのテープと、そこに記録された演算ひとつひとつに関連する勾配を使い、[トップダウン型自動微分（リバースモード）](https://ja.wikipedia.org/wiki/自動微分)を使用して、「記録」された計算の勾配を計算します。

例を示します。

In [0]:
x = tf.ones((2, 2))
  
with tf.GradientTape() as t:
  t.watch(x)
  y = tf.reduce_sum(x)
  z = tf.multiply(y, y)

# 元の入力テンソル x に対する z の微分
dz_dx = t.gradient(z, x)
for i in [0, 1]:
  for j in [0, 1]:
    assert dz_dx[i][j].numpy() == 8.0

「記録された」`tf.GradientTape` のコンテキスト中の、中間の値に対する出力の勾配を求めることもできます。

In [0]:
x = tf.ones((2, 2))
  
with tf.GradientTape() as t:
  t.watch(x)
  y = tf.reduce_sum(x)
  z = tf.multiply(y, y)

# テープを使って中間値 y に対する z の微分を計算
dz_dy = t.gradient(z, y)
assert dz_dy.numpy() == 8.0

デフォルトでは、ある GrandientTape に保持されたリソースは、GradientTape.gradient() メソッドが呼び出されると解放されます。一つの計算で複数の勾配を計算したい場合には、永続的(`persistent`)な勾配テープを作成します。こうすると、リソースはテープオブジェクトがガベージコレクションによって廃棄されるときに解放されるため、`gradient()` メソッドを複数回呼び出すことができます。  

例を示します。

In [0]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
  t.watch(x)
  y = x * x
  z = y * y
dz_dx = t.gradient(z, x)  # 108.0 (4*x^3 at x = 3)
dy_dx = t.gradient(y, x)  # 6.0
del t  # テープへの参照を削除

### 制御フローの記録

勾配テープは演算を実行の都度記録するため、（たとえば `if` や `while` を使った）Python の制御フローも自然に扱われます。

In [0]:
def f(x, y):
  output = 1.0
  for i in range(y):
    if i > 1 and i < 5:
      output = tf.multiply(output, x)
  return output

def grad(x, y):
  with tf.GradientTape() as t:
    t.watch(x)
    out = f(x, y)
  return t.gradient(out, x) 

x = tf.convert_to_tensor(2.0)

assert grad(x, 6).numpy() == 12.0
assert grad(x, 5).numpy() == 12.0
assert grad(x, 4).numpy() == 4.0

### 高次勾配

`GradientTape` コンテキストマネージャの内側で行われた演算は、自動微分のために記録されます。このコンテキスト内で勾配が計算された場合、勾配計算も同様に記録されます。結果として、まったくおなじAPIが高次勾配にも使えることになります。例を示します。

In [0]:
x = tf.Variable(1.0)  # 1.0 で初期化された TensorFlow 変数を作成

with tf.GradientTape() as t:
  with tf.GradientTape() as t2:
    y = x * x * x
  # ’t’ コンテキストマネジャー内で勾配を計算
  # これは勾配計算も同様に微分可能であるということ
  dy_dx = t2.gradient(y, x)
d2y_dx2 = t.gradient(dy_dx, x)

assert dy_dx.numpy() == 3.0
assert d2y_dx2.numpy() == 6.0

## 次のステップ

このチュートリアルでは、TensorFlow の勾配計算を扱いました。これで、ニューラルネットワークの構築と訓練のために必要な構成要素が揃いました。